#BLAST Homology Search Pipeline
## TEAM MEMBERS:
Harini C.S,
Roliza J.G,
Anisha.J,
Libarna.G,
Padma Amruthathika.A
Manisha.M.K
Fathima Haffitha M.F,
Naseeha Ahamed Ali,
Jegatheeshwari.K,
Vibisha V.K,
Athira.R,
Dhusyanthi.T

##INTRODUCTION
  The BLAST Homology Search Pipeline is a bioinformatics workflow used to find regions of similarity between biological sequences.It uses the BLAST algorithm to compare a query sequence against a database of known sequence.This process helps researchers identify homologous sequences,infer function,or detect evolutionary relationships.  

In [1]:
#BLAST homology search pipeline

# Install Biopython
!pip install biopython

from Bio.Blast import NCBIWWW, NCBIXML
from Bio import SeqIO
import os

# 🔹 Step 1: Create a dummy FASTA file (protein or DNA)
# You can switch between protein and DNA by commenting/uncommenting

# Dummy protein sequence
dummy_fasta = """>dummy_protein
MTEITAAMVKELRESTGAGMMDCKNALSETQHEKHHG"""

# Dummy nucleotide sequence
# dummy_fasta = """>dummy_dna
# ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG"""

# Save to file
with open("dummy.fasta", "w") as f:
    f.write(dummy_fasta)

query_file = "dummy.fasta"

# 🔹 Step 2: Read and inspect the sequence
record = SeqIO.read(query_file, format="fasta")
seq = str(record.seq).upper()

# Detect sequence type
is_dna = set(seq) <= set("ACGTUN")
seq_type = "nucleotide" if is_dna else "protein"
print(f"Detected sequence type: {seq_type}")

# 🔹 Step 3: Ask user for BLAST type (automated if protein)
if seq_type == "nucleotide":
    print("Available BLAST types: blastn (nucleotide), blastx (translated protein)")
    blast_program = input("Enter BLAST program (blastn/blastx): ").strip().lower()
    if blast_program not in ["blastn", "blastx"]:
        print("Invalid choice. Using default: blastn")
        blast_program = "blastn"
    db = "nt" if blast_program == "blastn" else "nr"
else:
    blast_program = "blastp"
    db = "nr"
    print("Protein sequence detected. Using BLASTp.")

# 🔹 Step 4: Run BLAST
print(f"Running {blast_program} against {db} for sequence ID: {record.id}")
result_handle = NCBIWWW.qblast(blast_program, db, record.seq)

# Save XML result
xml_output_file = "blast_result.xml"
with open(xml_output_file, "w") as out_handle:
    out_handle.write(result_handle.read())
result_handle.close()
print(f"BLAST results saved to {xml_output_file}")

# 🔹 Step 5: Parse and filter results
with open(xml_output_file) as result_handle:
    blast_record = NCBIXML.read(result_handle)

# E-value filter
e_value_thresh = 0.001
hits = []
for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        if hsp.expect < e_value_thresh:
            hits.append({
                "title": alignment.title,
                "accession": alignment.accession,
                "score": hsp.score,
                "e_value": hsp.expect,
                "identities": hsp.identities,
                "align_length": alignment.length
            })

# 🔹 Step 6: Display top hits
print(f"\nTop hits (E-value < {e_value_thresh}):\n")
for i, hit in enumerate(hits[:10], 1):
    print(f"{i}. {hit['title']}")
    print(f"   Accession: {hit['accession']}, Score: {hit['score']}, E-value: {hit['e_value']}")
    print(f"   Identities: {hit['identities']} / {hit['align_length']}\n")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 19.4 MB/s eta 0:00:00
Detected sequence type: protein
Protein sequence detected. Using BLASTp.
Running blastp against nr for sequence ID: dummy_protein
BLAST results saved to blast_result.xml

Top hits (E-value < 0.001):

1. gb|EOI8813943.1| elongation factor Ts [Campylobacter jejuni]
   Accession: EOI8813943, Score: 151.0, E-value: 2.29588e-11
   Identities: 30 / 62

2. gb|EAJ7469740.1| elongation factor Ts [Campylobacter jejuni] >gb|ECP7232318.1| elongation factor Ts [Campylobacter jejuni]
   Accession: EAJ7469740, Score: 148.0, E-value: 9.88172e-11
   Identities: 29 / 75

3. gb|EAL9427858.1| elongation factor Ts [Campylobacter jejuni]
   Accession: EAL9427858, Score: 147.0, E-value: 1.00382e-10
   Identities: 29 / 64

4. ref|WP_158334062.1| translation elongation factor Ts [Campylobacter bilis] >gb|MBM0636689.1| elongation factor Ts [Campylobacter bilis] >gb|MCC8277533.1| translation elongation factor Ts [Campylobacter bilis] >g